In [2]:
from fastai.text.all import *
import polars as pl

In [3]:
pl.Config.set_fmt_str_lengths(200);

# Data

In [4]:
labels_dict = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
df = (
    pl.concat([
        pl.scan_csv('emotions-dataset/training.csv').with_columns(is_valid=False),
        pl.scan_csv('emotions-dataset/validation.csv').with_columns(is_valid=True),
    ])
    .with_columns(pl.col.label.replace_strict(labels_dict).alias('label_name'))
    .collect()
)
print(df.shape)
df.head()

(18000, 4)


text,label,is_valid,label_name
str,i64,bool,str
"""i didnt feel humiliated""",0,false,"""sadness"""
"""i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake""",0,false,"""sadness"""
"""im grabbing a minute to post i feel greedy wrong""",3,false,"""anger"""
"""i am ever feeling nostalgic about the fireplace i will know that it is still on the property""",2,false,"""love"""
"""i am feeling grouchy""",3,false,"""anger"""


## Part 1 - LLM

In [6]:
dls = TextDataLoaders.from_df(df.to_pandas(), text_col='text', is_lm=True, valid_pct=0.1)

In [7]:
dls.show_batch(max_n=3)

,text,text_
0,xxbos i ve found it has made a huge difference especially on the finger with my ring and the my skin feels so much xxunk and less irritated xxbos i did not feel in the least smart xxbos i must find a way to accept these limitations until they are older without feeling held back or resentful xxbos i say that i feel like i m hated xxbos i m feeling shy,i ve found it has made a huge difference especially on the finger with my ring and the my skin feels so much xxunk and less irritated xxbos i did not feel in the least smart xxbos i must find a way to accept these limitations until they are older without feeling held back or resentful xxbos i say that i feel like i m hated xxbos i m feeling shy to
1,homework while i only got a c when we got the results he acted as if he did not xxunk this grade i found that his humility was xxunk and i found it disgusting xxbos i still did nt start feeling contractions but it was a tender mercy for me because she would have come on the st no matter what xxbos i feel kind of sorry for her xxbos i feel,while i only got a c when we got the results he acted as if he did not xxunk this grade i found that his humility was xxunk and i found it disgusting xxbos i still did nt start feeling contractions but it was a tender mercy for me because she would have come on the st no matter what xxbos i feel kind of sorry for her xxbos i feel like
2,view i thought you felt it too when there was me and you lyrics from a href http xxrep 3 w xxbos i feel offended i choose to tell you guys how i feel because i treated you guys as friends and would want to put a stop to all these nonsense xxbos i only feel curious impatient eager and confused xxbos i say this because she never truly gets a choice,i thought you felt it too when there was me and you lyrics from a href http xxrep 3 w xxbos i feel offended i choose to tell you guys how i feel because i treated you guys as friends and would want to put a stop to all these nonsense xxbos i only feel curious impatient eager and confused xxbos i say this because she never truly gets a choice or


### Model

In [9]:
learn = language_model_learner(dls, AWD_LSTM, metrics=accuracy, wd=0.1).to_fp16()

In [12]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(5600, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(5600, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0-2): 3 x RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=5600, bias=True)
    (output_dp): RNNDropout()
  )
)

### Train

In [13]:
learn.fit_one_cycle(2, 1e-2)

/home/renato/github/pucp_bootcamp_202401/.venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/home/renato/github/pucp_bootcamp_202401/.venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,4.645377,4.109325,0.264740,00:06
1,4.308812,4.062562,0.271428,00:05


In [14]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

/home/renato/github/pucp_bootcamp_202401/.venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/home/renato/github/pucp_bootcamp_202401/.venv/lib/python3.12/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,4.098083,3.992083,0.279975,00:07
1,4.035848,4.055367,0.266022,00:07
2,3.933802,3.945851,0.290399,00:07
3,3.829961,3.945451,0.287987,00:07
4,3.698956,3.936899,0.290611,00:07
5,3.560176,3.954743,0.292875,00:07
6,3.443704,3.981243,0.292882,00:07
7,3.335767,4.003896,0.292552,00:06
8,3.264158,4.020332,0.291351,00:07
9,3.217096,4.025388,0.291519,00:07


In [15]:
learn.save_encoder('lm_encoder')

In [18]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [35]:
df.filter(pl.col.is_valid, pl.col.label_name=='joy').sample()

text,label,is_valid,label_name
str,i64,bool,str
"""i am feeling more and more eager to get on with my move""",1,true,"""joy"""


In [43]:
inp = 'I am feeling'
samples = 5
for _ in range(samples):
    res = learn.predict(inp, 30, temperature=0.75, no_bar=True)
    print(f'- {res}\n')

- i am feeling very weepy so i can not move i feel so doubtful that everything i ca nt do do nt see what i want to do next i feel

- i am feeling all apprehensive and sad about the whole thing i feel so reluctant to post on this blog i feel more outgoing and able to express my feelings

- i am feeling depressed about it i feel very appreciative of all the books we ve seen i feel pretty good because i think that the pain that i feel is

- i am feeling bitchy too i m feeling a bit groggy today i just feel so regretful when i cry i have a feeling it will be a terrific one

- i am feeling the need to capture the warmth of the sun to fill me inside i m feeling quite pleased with myself i am feeling very discontent i know



## Fine-tuning

In [46]:
dls = TextDataLoaders.from_df(df.to_pandas(), text_col='text', label_col='label_name', valid_col='is_valid')

In [47]:
dls.show_batch(max_n=3)

,text,category
0,xxbos i feel in my bones like nobody cares if i m here nobody cares if i m gone here i am again saying i m feeling so lonely people either say its ok to be alone or just go home it xxunk me and i do nt know why it does nt mean i do nt try i try and try but people just treat me like i m a xxunk,sadness
1,xxbos i confused my feelings with the truth because i liked the view when there was me and you i ca nt believe that i could be so blind its like you were floating when i was falling and i did nt mind because i like the view i thought you felt it too when there was me and you lyrics from a href http xxrep 3 w,love
2,xxbos i feel not offended in any form and should not make this big and in the end it does nt bother me at all but i ve learned to show some balls in the past and say what i think not xxunk so if we would give some weight to the content of these comments there would be the questions what is behind it,anger


### Model

In [49]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=accuracy).to_fp16()

In [50]:
learn.model

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(5600, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(5600, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0-2): 3 x RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): LinBnDrop(
        (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Dropout(p=0.2, inplace=False)
        (2): Linear(in_features=1200, out_features=50, bias=False)
        (3): ReLU(inplace=True)
      )
      (1): LinBnDrop(
        (0): Bat

In [51]:
learn.load_encoder('lm_encoder')

### Train

In [55]:
learn.fit_one_cycle(5, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,34.434536,nan,0.428000,00:03
1,8.441158,nan,0.481500,00:03
2,2.849279,nan,0.496500,00:03
3,2.082439,nan,0.507000,00:03
4,1.642406,nan,0.520500,00:03


In [56]:
learn.unfreeze()

In [57]:
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.400899,nan,0.685500,00:07
1,0.481857,0.221862,0.921000,00:07
2,0.296643,0.128695,0.946500,00:07
3,0.214064,0.145025,0.938500,00:07
4,0.178960,0.157566,0.931000,00:07
5,0.120854,0.144305,0.940500,00:07
6,0.107106,0.167099,0.947000,00:07
7,0.092814,0.268928,0.942500,00:07
8,0.088158,1.100440,0.939500,00:07
9,0.092189,0.217724,0.938000,00:07


In [59]:
learn.show_results()

,text,category,category_
0,xxbos i tell myself i do nt open my mouth and say what i really feel because i know i m a xxunk and i m smart enough to know i m a xxunk and i never ever know if what i m doing the choices i make are really what i want or need or even the right thing or if its the disease,joy,joy
1,xxbos i could nt hellip even when it made my heart ache to simply look at you hellip because i loved you so much and i knew you would never return my feelings hellip and i could nt bring myself to hate you for the idiotic xxunk you pulled in the other room either though i do ask that you do nt repeat it,sadness,sadness
2,xxbos i really want to watch it for the obvious romance reasons and i have a feeling like it would be a really funny kind of drama too i can also somewhat relate to the female main character who works with this boss who has a lot of pride and is a bit what you would like to call him as xxunk,surprise,surprise
3,xxbos i feel stressed out all the time i said and then i think about how people say stress causes cancer and i know it isn t true but i can t stop thinking that i need to relax or else my cancer will come back and then i get stressed out because i m stressed and it makes me feel worse,sadness,sadness
4,xxbos i could go on and on right now about what we ve been through this year and what i ve learned what xxunk could do when and such but i wo nt because this would be a book and honestly i m not feeling fabulous today and xxunk has been dealing with a giant cold since thursday and we are wiped,joy,joy
5,xxbos i just feel you so so don t be afraid i should hurt even more and pray again so i can find you again the more time passes the more it hurts i need you go back in time just one time forgive my sins if only i could turn things back this pain would be so so sloth,fear,fear
6,xxbos i know my xxunk is stronger than my behaviour over the weekend and i need to focus on the joy and health that all the great food i brought with me gives and how i could ve if i really wanted to indulge indulged in that great stuff i know its not the same but i would feel amazing,joy,joy
7,xxbos i have been out there over the last few weeks i experienced for the first time a feeling of loving the actual act of running of pushing my daughter in the xxunk of getting xxunk by my wife although this would happen if i was in top shape anyway of having cold air nail you in the face,love,love
8,xxbos i feel so strong and i find a new way you never come back and i try to stay on the sunny side of life and i know that i will forget you i feel it deep in my heart no matter that you never loved me i do not regret the separation i welcome the new start,joy,joy


In [60]:
interp = Interpretation.from_learner(learn)

In [61]:
interp.plot_top_losses(5)

,input,target,predicted,probability,loss
0,xxbos i popped a fever and even my co workers we xxunk me to go home before i even had a chance to open my mouth and voice the xxunk i m not feeling so hot,love,anger,0.9278222322463989,67.29366302490234
1,xxbos i guess i feel betrayed because i admired him so much and for someone to do this to his wife and kids just goes beyond the pale,joy,love,0.9989461302757263,8.877607345581055
2,xxbos i had finished my first leg the xxunk longest and xxunk one of my three but i was not feeling so hot,love,joy,0.9157993197441101,8.532682418823242
3,xxbos i feel not having a generous spirit or a forgiving nature xxunk me off from accepting gifts from the universe,joy,love,0.9997885823249817,7.586617469787598
4,xxbos xxunk a xxunk race at a xxunk xxunk with other friends,joy,anger,0.9974024891853333,7.537271022796631


In [62]:
learn.predict('Write here some random phrase here', with_input=True)

('xxbos xxmaj write here some random phrase here',
 'fear',
 tensor(1),
 tensor([0.1460, 0.4132, 0.2302, 0.0009, 0.1960, 0.0137]))